In [8]:
from adlfs import AzureBlobFileSystem
import duckdb
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import os

ACC_KEY = os.environ["ACC_KEY"]
ACC_NAME = os.environ["ACC_NAME"]
CONTAINER_NAME = os.environ["CONTEINER"]
FOLDER = os.environ["FOLDER"]
FILE_NAME = os.getenv("AZURE_FILE_NAME")

path = f"abfs://{CONTAINER_NAME}/{FOLDER}/{FILE_NAME}"

abfs = AzureBlobFileSystem(
    account_name = ACC_NAME,
    account_key = ACC_KEY,
    container_name = CONTAINER_NAME)

pqdata = ds.dataset(path, filesystem=abfs)

conn = duckdb.connect(":memory:")
conn.sql("SELECT * from pqdata").show()

┌───────────────────┬──────────────────────────┐
│     carga_mw      │           data           │
│      double       │ timestamp with time zone │
├───────────────────┼──────────────────────────┤
│       517292.1985 │ 2020-12-31 21:00:00-03   │
│       663260.0435 │ 2021-01-01 21:00:00-03   │
│       656525.3775 │ 2021-01-02 21:00:00-03   │
│       796937.2295 │ 2021-01-03 21:00:00-03   │
│        875973.059 │ 2021-01-04 21:00:00-03   │
│        915011.111 │ 2021-01-05 21:00:00-03   │
│       910762.1215 │ 2021-01-06 21:00:00-03   │
│       888382.2145 │ 2021-01-07 21:00:00-03   │
│       832799.8905 │ 2021-01-08 21:00:00-03   │
│       751290.1805 │ 2021-01-09 21:00:00-03   │
│            ·      │           ·              │
│            ·      │           ·              │
│            ·      │           ·              │
│ 868344.0109999999 │ 2024-08-04 21:00:00-03   │
│ 908806.9670000001 │ 2024-08-05 21:00:00-03   │
│        923834.382 │ 2024-08-06 21:00:00-03   │
│       932343.3125 

In [14]:
# number of days between two dates
import pandas as pd

df = pqdata.to_table().to_pandas().reset_index()
df


,data,carga_mw
0,2021-01-01 00:00:00+00:00,517292.1985
1,2021-01-02 00:00:00+00:00,663260.0435
2,2021-01-03 00:00:00+00:00,656525.3775
3,2021-01-04 00:00:00+00:00,796937.2295
4,2021-01-05 00:00:00+00:00,875973.0590
...,...,...
1317,2024-08-10 00:00:00+00:00,795148.5160
1318,2024-08-11 00:00:00+00:00,698073.5950
1319,2024-08-12 00:00:00+00:00,805498.2440
1320,2024-08-13 00:00:00+00:00,844564.0770


In [28]:
max_data = df["data"].max()
min_data = df["data"].min()

delta = max_data - min_data

print(f"There are {delta.days + 1} days between {min_data.date()} and {max_data.date()}")

There are 1322 days between 2021-01-01 and 2024-08-14
